In [81]:
import numpy as np
import glob
import random
import struct
from tensorflow.core.example import example_pb2
import tensorflow as tf

In [39]:
data_folder = "/content/drive/MyDrive/Universiteit Utrecht/"\
"Business Informatics/Thesis Project/Implementation/Data/CNN Daily Mail/"\
"non_anonymized/"

In [155]:
# https://github.com/abisee/pointer-generator/blob/master/data.py
def example_generator(data_path, single_pass=False):
  """Generates tf.Examples from data files.
    Binary data format: <length><blob>. <length> represents the byte size
    of <blob>. <blob> is serialized tf.Example proto. The tf.Example contains
    the tokenized article text and summary.
  Args:
    data_path:
      Path to tf.Example data files. Can include wildcards, e.g. if you have 
      several training data chunk files train_001.bin, train_002.bin, etc, 
      then pass data_path=train_* to access them all.
    single_pass:
      Boolean. If True, go through the dataset exactly once, generating 
      examples in the order they appear, then return. Otherwise, generate 
      random examples indefinitely.
  Yields:
    Deserialized tf.Example.
  """
  while True:
    # get the list of datafiles
    filelist = glob.glob(data_path)
    # check filelist isn't empty
    assert filelist, ('Error: Empty filelist at %s' % data_path)
    if single_pass:
      filelist = sorted(filelist)
    else:
      random.shuffle(filelist)
    for f in filelist:
      reader = open(f, 'rb')
      while True:
        len_bytes = reader.read(8)
        # finished reading this file
        if not len_bytes: break
        str_len = struct.unpack('q', len_bytes)[0]
        example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
        yield example_pb2.Example.FromString(example_str)
    if single_pass:
      print("example_generator completed reading all datafiles. No more data.")
      break

In [4]:
# https://github.com/abisee/pointer-generator/blob/master/batcher.py
def text_generator(self, example_generator):
    """Generates article and abstract text from tf.Example.
    Args:
      example_generator: a generator of tf.Examples from file. 
      See data.example_generator"""
    while True:
      # e is a tf.Example
      e = example_generator.next()
      try:
        # the article text was saved under the key 'article' in the data files
        article_text = e.features.feature['article'].bytes_list.value[0]
        # the abstract text was saved under the key 'abstract' in the data files
        abstract_text = e.features.feature['abstract'].bytes_list.value[0]
      except ValueError:
        tf.logging.error('Failed to get article or abstract from example')
        continue
      if len(article_text)==0: # See https://github.com/abisee/pointer-generator/issues/1
        tf.logging.warning('Found an example with empty article text. Skipping it.')
      else:
        yield (article_text, abstract_text)

In [90]:
train_000_file = data_folder+"chunked/train_*.bin"
train_000 = example_generator(train_000_file)

In [154]:
count = 0
str_lennie = 0
for example in train_000:
  #print(example["abstract"])
  print(str_lennie)
  print(example)
  break
  
#print(f"There are {count} items in {train_000_file}")

6991
b"\n\xcc6\n\xb9\x02\n\x08abstract\x12\xac\x02\n\xa9\x02\n\xa6\x02<s> the ibm supercomputer is set to become a customer service manager called the watson engagement advisor . </s> <s> customers will be able to complain to watson through phone , email , instant messaging . </s> <s> watson-powered smartphone app , ` ask watson ' expected later this year . </s>\n\x8d4\n\x07article\x12\x814\n\xfe3\n\xfb3technology company ibm is putting its supercomputer watson to work - in a robot call centre . the artificially intelligent computer system is taking on the role of customer service manager and will be called watson engagement advisor . companies will be able to sign up to ibm 's service and its customers can then ring a helpline and complain or get help from the question answering -lrb- qa -rrb- machine . the ibm watson mobile developer challenge invited teams to design mobile apps using watson 's , pictured , cognitive computing capabilities . the 25 finalists will now submit prototype

In [87]:
reader = open(train_000_file, 'rb')
while True:
    len_bytes = reader.read(8)
    # finished reading this file
    if not len_bytes: break
    str_len = struct.unpack('q', len_bytes)[0]
    print(str_len)
    example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
    print(example_str)
    break

4569
b"\n\xd6#\n\xdf\x02\n\x08abstract\x12\xd2\x02\n\xcf\x02\n\xcc\x02<s> mentally ill inmates in miami are housed on the `` forgotten floor '' </s> <s> judge steven leifman says most are there as a result of `` avoidable felonies '' </s> <s> while cnn tours facility , patient shouts : `` i am the son of the president '' </s> <s> leifman says the system is unjust and he 's fighting for change . </s>\n\xf1 \n\x07article\x12\xe5 \n\xe2 \n\xdf editor 's note : in our behind the scenes series , cnn correspondents share their experiences in covering news and analyze the stories behind the events . here , soledad o'brien takes users inside a jail where many of the inmates are mentally ill . an inmate housed on the `` forgotten floor , '' where many mentally ill inmates are housed in miami before trial . miami , florida -lrb- cnn -rrb- -- the ninth floor of the miami-dade pretrial detention facility is dubbed the `` forgotten floor . '' here , inmates with the most severe mental illnesses are

In [89]:
from google.protobuf.json_format import MessageToJson

reader = open(train_000_file, 'rb')
while True:
    len_bytes = reader.read(8)
    # finished reading this file
    if not len_bytes: break
    str_len = struct.unpack('q', len_bytes)[0]
    print(str_len)
    example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
    print(MessageToJson(tf.train.Example.FromString(example_str)))
    break

4569
{
  "features": {
    "feature": {
      "article": {
        "bytesList": {
          "value": [
            "ZWRpdG9yICdzIG5vdGUgOiBpbiBvdXIgYmVoaW5kIHRoZSBzY2VuZXMgc2VyaWVzICwgY25uIGNvcnJlc3BvbmRlbnRzIHNoYXJlIHRoZWlyIGV4cGVyaWVuY2VzIGluIGNvdmVyaW5nIG5ld3MgYW5kIGFuYWx5emUgdGhlIHN0b3JpZXMgYmVoaW5kIHRoZSBldmVudHMgLiBoZXJlICwgc29sZWRhZCBvJ2JyaWVuIHRha2VzIHVzZXJzIGluc2lkZSBhIGphaWwgd2hlcmUgbWFueSBvZiB0aGUgaW5tYXRlcyBhcmUgbWVudGFsbHkgaWxsIC4gYW4gaW5tYXRlIGhvdXNlZCBvbiB0aGUgYGAgZm9yZ290dGVuIGZsb29yICwgJycgd2hlcmUgbWFueSBtZW50YWxseSBpbGwgaW5tYXRlcyBhcmUgaG91c2VkIGluIG1pYW1pIGJlZm9yZSB0cmlhbCAuIG1pYW1pICwgZmxvcmlkYSAtbHJiLSBjbm4gLXJyYi0gLS0gdGhlIG5pbnRoIGZsb29yIG9mIHRoZSBtaWFtaS1kYWRlIHByZXRyaWFsIGRldGVudGlvbiBmYWNpbGl0eSBpcyBkdWJiZWQgdGhlIGBgIGZvcmdvdHRlbiBmbG9vciAuICcnIGhlcmUgLCBpbm1hdGVzIHdpdGggdGhlIG1vc3Qgc2V2ZXJlIG1lbnRhbCBpbGxuZXNzZXMgYXJlIGluY2FyY2VyYXRlZCB1bnRpbCB0aGV5ICdyZSByZWFkeSB0byBhcHBlYXIgaW4gY291cnQgLiBtb3N0IG9mdGVuICwgdGhleSBmYWNlIGRydWcgY2hhcmdlcyBvciBjaGFyZ2VzIG9mIGFz